<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Данные нужно защитить так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

**Цель проекта:** Необходимо преобразовать данные, чтобы по ним было сложно восстановить персональную информацию, и, при этом, сохранить качество моделей машинного обучения на прежнем уровне.

**Описание данных:** Датасет с данными клиентов страховой компании: пол, возраст, зарплата застрахованного, количество членов семьи и количество страховых выплат за последние пять лет.

## Загрузка данных

In [1]:
# Импортируем необходимые библиотеки:

import numpy as np
import pandas as pd
import scipy
from sklearn.metrics import r2_score

In [2]:
# Считаем данные из csv-файла в датафрейм, сохраним в переменную и выведем ее на экран:

data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
# Изучим основную информацию о датафрейме:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Обоснование:**

Используемые свойства:
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$
Доказательство:
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
Требуется доказать, что предсказания не изменятся, имеем  $a =  Xw$,   $a' = X'w'$
\
\
$$
    w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$
\
Тогда $w' = P^{-1} w$, следовательно:
\
\
$$
a' = (XP)w' = XPP^{-1}w = Xw = a
$$
чтд.

In [4]:
# Сгенерируем случайную матрицу, размер которой равен количеству
# признаков исходного датасета, и выведем ее на экран:

arr = np.random.normal(size = (4,4 ))
arr

array([[-0.13130559, -0.41746164,  2.3502439 ,  0.15705539],
       [ 0.55435138, -0.81452195,  0.10779294,  1.68618947],
       [ 0.47191128,  0.33767746,  1.55967145, -1.43748666],
       [ 0.13771695,  0.08060308, -1.23567805,  1.06837948]])

In [5]:
# Найдем обратную ей матрицу:

arr_rev = np.linalg.inv(arr)
arr_rev

array([[-0.55616488,  0.73505103,  1.01642677,  0.28923553],
       [ 0.93948481, -1.11634685,  0.78907386,  2.68547305],
       [ 0.52097141, -0.14529138,  0.19463608,  0.4146038 ],
       [ 0.60336323, -0.17857078,  0.03456314,  1.17563732]])

In [6]:
# Проверим, что их скалярное произведение равно единичной матрице
# (с учетом погрешности):

print(arr @ arr_rev)

[[ 1.00000000e+00  1.05008636e-16 -2.36784695e-17  4.70157477e-17]
 [ 4.18011834e-17  1.00000000e+00  6.34094176e-17 -2.75669776e-17]
 [ 5.60092079e-16 -3.60783696e-16  1.00000000e+00  7.85488927e-16]
 [-1.14669620e-16  2.84735669e-17 -5.74934806e-17  1.00000000e+00]]


**Ответ:** При перемножении признаков на обратимую матрицу, качество линейной регрессии не изменится.

**Обоснование:** Предсказание модели вычисляется, как скалярное произведение матрицы признаков Х на вектор весов w, а длина вектора признаков, в свою очередь, определяется количеством признаков в матрице Х. Соответственно, при перемножении матрицы Х на обратимую матрицу, длина вектора будет зависить уже от полученной новой матрицы.

## Алгоритм преобразования

**Алгоритм**

1) Выделить из исходного датасета признаки и целевой признак.

2) Запрограммировать линейную регрессию, обучить ее на основании исходных признаков и получить предсказания; определить значение R2

3) Функцией numpy.random.normal() сгенерировать случайную матрицу, размер которой будет соответствовать количеству признаков исходного датасета.

4) Найти обратную полученной матрицу, вызвав функцию numpy.linalg.inv()

5) Преобразовать исходные признаки в матрицу и, получив ее произведение на обратимую, преобразовать обратно в датасет.

6) Обучить линейную регрессию на основании новых признаков, получить предсказания и значение R2; сверить его с предыдущим.

**Обоснование**

Поскольку предсказание модели вычисляется, как скалярное произведение матрицы признаков Х на вектор весов w, а длина вектора признаков, в свою очередь, определяется количеством признаков в матрице Х, то при перемножении матрицы Х на обратимую матрицу, качество линейной регрессии будет таким же, так как длина вектора остается неизменной.

## Проверка алгоритма

In [7]:
# Выделим из исходного датасета признаки и целевой признак:

features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [8]:
# Запрограммируем модель линейной регрессии, обучим ее, получим предсказания и
# значение метрики R2:

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = ((np.linalg.inv(X.T.dot(X))).dot(X.T)).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4249455028666801


In [9]:
# Создадим список с названиями признаков:

features_columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи']

In [10]:
# Преобразуем датасет с признаками в матрицу, перемножим на обратимую матрицу и
# вновь преобразуем полученный результат в датасет признаков:

features_tf = pd.DataFrame(features.values @ arr_rev, columns=features_columns)

In [11]:
# Выведем преобразованные признаки на экран:

print(features_tf.head())

            Пол      Возраст     Зарплата   Члены семьи
0  25878.747822 -7251.666129  9687.352586  20675.917981
1  19840.733100 -5572.602920  7432.503001  15879.651976
2  10967.644589 -3083.493013  4110.240822   8784.558617
3  21745.443545 -6082.450921  8132.964213  17347.724865
4  13623.103111 -3822.627647  5103.112183  10896.641784


In [12]:
# Обучим модель на основании новых признаков, получим предсказания и значение R2:

model.fit(features_tf, target)
predictions = model.predict(features_tf)
print(r2_score(target, predictions))

0.42494550286667687


**Вывод:** Значение метрики R2 как до, так и после преобразования, составило приблизительно 0,425.

## Общий вывод

В ходе работы над проектом была запрограммирована и обучена на основании признаков исходного датасета модель линейной регрессии. В результате, получены предсказания и определено значение метрики R2.

Затем был написан алгоритм преобразования данных для решения задачи. В результате его применения:

 - сгенерирована случайная матрица;

 - найдена матрица, обратная полученной;

 - исходные признаки перемножены на обратимую матрицу;

 - произведено обучение модели линейной регрессии на основании новых признаков, получены предсказания и значение R2.

Сформулирован следующий вывод: поскольку предсказание модели вычисляется, как скалярное произведение матрицы признаков Х на вектор весов w, а длина вектора признаков, в свою очередь, определяется количеством признаков в матрице Х, то при перемножении матрицы Х на обратимую матрицу, качество линейной регрессии будет таким же, так как длина вектора остается неизменной.

В нашем случае, значение метрики R2 как до, так и после преобразования, составило приблизительно 0,425.